# Final Project: Admission Prediction from NHAMCS
## Initial LR model
### DS5559: Big Data Analysis
### Thomas Hartka, Alicia Doan, Michael Langmayr
Created: 6/28/2020 
  
In this notebook creates and analyzes logistic regression model for predicting hospital admission in the NHAMCS database.  For this iniital model, categorical variables will be represented with one-hot encoding and the reason for visit (RFV) variables will be ignored.  The RFV varaibles will be ignored because there are hundreds of different potential values and we have not yet developed a way to categorize them yet.  Binary variables were previously converted to 0/1.  

## Configuration

In [1]:
# preferences
scale_data = True        # should data be scaled
weight_outcome = True    # use weights to handle class imbalance
reg_param = 0            # regularization of LR (0=not regularizatoin)
elas_param = 0           # elastic net (0=Ridge,1=Lasso)
reduced_vars = False     # where to use a reduced variable set

In [2]:
# set data directory
data_dir = "../data"
results_dir = "../results"

## Import libraries and set up Spark

In [3]:
# import python libraries
import os
import pandas as pd
import numpy as np
from functools import reduce

In [4]:
# set up pyspark
from pyspark.sql import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

In [5]:
from pyspark.ml import Pipeline  
from pyspark.ml.feature import *  
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [6]:
spark = SparkSession.builder.getOrCreate()

## Read in data

In [7]:
NHAMCS = spark.read.parquet(data_dir + "/NHAMCS_processed.2014-2017")

## Transform data

In [8]:
# perform string indexing to prepare for OHE for residence variable
rsi = StringIndexer(inputCol="RESIDNCE", outputCol="RESINDEX")
simodel = rsi.fit(NHAMCS)
NHAMCS = simodel.transform(NHAMCS)

In [9]:
# perform OHE on residence variable
rohe = OneHotEncoder(inputCol='RESINDEX', outputCol='RESONE')
NHAMCS = rohe.transform(NHAMCS)

In [10]:
# assemble vector
if reduced_vars:
    va = VectorAssembler(inputCols=['AGEYEAR','PULSE','TEMPF','COPD'], 
                         outputCol="features") 
else:    
    va = VectorAssembler(inputCols=["AGEYEAR","RESONE",'SEXMALE','ARRTIMEMIN','YEAR','PULSE','TEMPF', \
                                'RESPR','BPSYS','BPDIAS','POPCT','PAINSCALE','ALZHD','ASTHMA','CAD','CANCER', \
                                'CEBVD','CHF','CKD','COPD','DEPRN','DIABTYP0','DIABTYP1','DIABTYP2','EDHIV', \
                                'ESRD','ETOHAB','HPE','HTN','HYPLIPID','OBESITY','OSA','OSTPRSIS','SUBSTAB', \
                                'NOCHRON','TOTCHRON','INJURY','INJURY72'], 
                         outputCol="features")  
    
NHAMCS = va.setHandleInvalid("skip").transform(NHAMCS)

In [11]:
# scale features using MaxAbs
scaler = MaxAbsScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(NHAMCS)
NHAMCS = scalerModel.transform(NHAMCS)

# determine feature column
if scale_data:
    features_col = "features"
else:
    features_col = "scaledFeatures"

## Train and test model

In [12]:
# split into training and testing set
training, testing = NHAMCS.randomSplit([0.8, 0.2], 42)

In [13]:
# handle class imbalance

# calculate balance ratio
balRatio = training.select("ADM_OUTCOME").where('ADM_OUTCOME == 0').count() / training.count()

# add weights
training = training.withColumn("classWeights", when(training.ADM_OUTCOME == 1,balRatio).otherwise(1-balRatio))

In [14]:
# set up LR model
if weight_outcome:
    lr = LogisticRegression(featuresCol=features_col, labelCol="ADM_OUTCOME", weightCol="classWeights", \
                              maxIter=10, regParam=reg_param, elasticNetParam=elas_param)
else:
    lr = LogisticRegression(featuresCol="features", labelCol="ADM_OUTCOME", \
                               maxIter=10, regParam=reg_param, elasticNetParam=elas_param)

# Fit the model
admModel = lr.fit(training)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Coefficients: " + str(admModel.coefficients))
print("Intercept: " + str(admModel.intercept))

Coefficients: [0.006959770135108006,-0.23376222712865802,0.3249245722128116,0.037676263227828556,0.22388186897546347,0.11175730200244996,0.06450360958845103,-0.06563847851127284,-5.6052702401406886e-05,0.00024461868131159524,0.0018773758707992221,-0.004419383705960031,0.007341971992478976,0.0017301654431197643,-0.001161088460256079,-0.00815454619899241,-0.002121602711287132,0.3840681176024984,0.015098226790087188,0.3324583964381161,0.34026970001041323,0.3359043662680101,0.38996814849133415,0.3777295837760766,0.29188987935721916,0.2738695189552391,0.12568245042105142,0.10098830381398406,0.2674776751632989,0.08025629166380828,0.41605451077349964,0.1820511442832238,0.33799798709698375,0.2926910719105972,0.24991722302869254,0.19786774755056852,0.28223288303841737,0.3188301052722539,0.18214335256793304,-0.353012813406735,0.1059442780428311,0.0,-0.0589793036351345]
Intercept: -6.426063178358598e-07


In [15]:
predict_train=admModel.transform(training)
predict_test=admModel.transform(testing)
predict_test.select("ADM_OUTCOME","rawPrediction","prediction","probability").show(10)

+-----------+--------------------+----------+--------------------+
|ADM_OUTCOME|       rawPrediction|prediction|         probability|
+-----------+--------------------+----------+--------------------+
|          1|[0.87307347316225...|       0.0|[0.70538482114152...|
|          0|[0.85828354494867...|       0.0|[0.70230191298937...|
|          0|[0.85746478597929...|       0.0|[0.70213070348003...|
|          0|[-0.1022590826539...|       1.0|[0.47445748346260...|
|          0|[0.80900795801750...|       0.0|[0.69189806588998...|
|          0|[0.92426358055808...|       0.0|[0.71591004254549...|
|          0|[1.01617639854912...|       0.0|[0.73422713941140...|
|          0|[1.00509089951636...|       0.0|[0.73205833206064...|
|          0|[0.96817841667667...|       0.0|[0.72475626874425...|
|          0|[0.89816292881166...|       0.0|[0.71057183768119...|
+-----------+--------------------+----------+--------------------+
only showing top 10 rows



## Evaluate model

In [16]:
evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="ADM_OUTCOME")

print("Number of negatives: ", predict_test.where('prediction == 0').count())
print("Number of positives: ", predict_test.where('prediction == 1').count())

print("\nThe area under ROC for train set is", evaluator.evaluate(predict_train))
print("The area under ROC for test set is", evaluator.evaluate(predict_test))

Number of negatives:  1513
Number of positives:  691

The area under ROC for train set is 0.7341131404078137
The area under ROC for test set is 0.7540790183387306
